In [1]:
%reload_ext autoreload
%autoreload 2

%cd /Users/chompk.visai/Works/cdao/connext/connext-galxe-analytics/

/Users/chompk.visai/Works/cdao/connext/connext-galxe-analytics


In [2]:
import logging
from datetime import datetime, timedelta
from typing import Dict, List, Optional

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from tqdm.auto import tqdm

from api.connext import ConnextAPI, ConnextLPTransferAPI
from api.contract import SmartContract
from api.constant import Chain
from api.token import Token
from api.price import WETHPriceFetcher
load_dotenv(".env")

True

## Define Functions

In [3]:
logging.basicConfig(level=logging.INFO)


def unix_to_datetime(ts: int) -> datetime:
    return datetime.utcfromtimestamp(int(ts))
    

def get_abis():
    erc20_abi = Token.address_mapper[Chain.GNOSIS]["canonical"][Token.USDC].abi
    diamond_abi = ConnextAPI().scan_api[Chain.GNOSIS].diamond_contract.abi
    return erc20_abi + diamond_abi


def get_topic_resolver():
    diamond = ConnextAPI().scan_api[Chain.GNOSIS].diamond_contract
    
    topic2sig = {}
    for abi in get_abis():
        name = abi["name"]
        inputs = ",".join([_item["type"] for _item in abi["inputs"]])
        sig = f"{name}({inputs})"
        topic_id = diamond.provider.toHex(diamond.provider.keccak(text=sig))

        topic2sig[topic_id] = [name] + [_item["type"] for _item in abi["inputs"]]
    return topic2sig


def resolve_address(address: str) -> str:
    """ensure padding to length of 40 (excluded 0x)"""
    hex_code = address.replace("0x", "")
    if len(hex_code) < 40:
        n_zeros = 40 - len(hex_code)
        hex_code = "0" * n_zeros + hex_code
    resolved_address = "0x" + hex_code
    assert len(resolved_address) == 42
    return resolved_address


def get_lp_txs(
    data: pd.DataFrame,
    transfers: pd.DataFrame,
    chains: List[Chain] = [Chain.POLYGON, Chain.ARBITRUM_ONE, Chain.BNB_CHAIN, Chain.GNOSIS, Chain.OPTIMISM],
    filter_topics: List[str] = ["Transfer"],
    filter_function: List[str] = ["addSwapLiquidity", "removeSwapLiquidity"],
    blacklist_token: List[str] = [],) -> pd.DataFrame:
    """Get txs involved with add/remove stable liquidity"""
    lp_txs = []
    topic2sig = get_topic_resolver()

    # iterate over chains that support liquidity providing
    for chain in chains:
        lp_tokens = [Token.get_lp(chain, _token).address.lower() for _token in [Token.USDC, Token.WETH]]
        provider = SmartContract.get_default_provider(chain)
        # iterate over txn
        for tx in data[chain]:
            # skip transactions that aren't liquidity providing related
            fn_name = tx.functionName.split("(")[0]
            if fn_name not in filter_function:
                continue

            for _log in tx.logs:
                topic, *topic_args = _log["topics"]
                topic_items = topic2sig.get(topic)

                # skip unknown topic name
                if topic_items is None:
                    continue

                topic_name, *topic_params = topic_items

                # filter unwanted topic
                if topic_name not in filter_topics:
                    continue

                # skip blacklist address
                if _log["address"].lower() in list(map(lambda x: x.lower(), blacklist_token)):
                    continue

                # for Transfer
                sender, receiver = topic_args
                sender = hex(int(sender, 16))
                receiver = hex(int(receiver, 16))
                token = Token.address_lookup(_log["address"], chain)
                amount = int(_log["data"], 16) / (10**token.decimal)

                if sender == "0x0":
                    action = "mint"
                    user = receiver
                elif receiver == "0x0":
                    action = "burn"
                    user = sender
                else:
                    # let's focus on mint/burn count
                    # as criteria will be selected 
                    # based on CLP anyway
                    continue

                lp_txs.append({
                    "chain": chain,
                    "tx_hash": tx.hash,
                    "sender": resolve_address(sender),
                    "receiver": resolve_address(receiver),
                    "token": token.symbol,
                    "amount": amount,
                    "action": action,
                    "fn_name": fn_name,
                    "user": resolve_address(user),
                    "timestamp": int(tx.timeStamp)
                })
                
        for tx in transfers[chain]:
            # iterate over transfer logs
            for _log in tx.logs:
                topic, *topic_args = _log["topics"]
                topic_items = topic2sig.get(topic)

                # skip unknown topic name
                if topic_items is None:
                    continue

                topic_name, *topic_params = topic_items

                # filter unwanted topic
                if topic_name not in filter_topics:
                    continue

                # skip blacklist address
                if _log["address"].lower() not in lp_tokens:
                # if _log["address"].lower() in list(map(lambda x: x.lower(), blacklist_token)):
                    # skip if transfer aren't LP token
                    continue

                # for Transfer
                sender, receiver = topic_args
                sender = hex(int(sender, 16))
                receiver = hex(int(receiver, 16))
                token = Token.address_lookup(_log["address"], chain)
                if token is None:
                    continue
                amount = int(_log["data"], 16) / (10**token.decimal)

                lp_txs.append({
                    "chain": chain,
                    "tx_hash": tx.hash,
                    "sender": resolve_address(sender),
                    "receiver": resolve_address(receiver),
                    "token": token.symbol,
                    "amount": amount,
                    "action": "transfer_out",
                    "fn_name": "Transfer",
                    "user": resolve_address(sender),
                    "timestamp": int(tx.timeStamp)
                })

                lp_txs.append({
                    "chain": chain,
                    "tx_hash": tx.hash,
                    "sender": resolve_address(sender),
                    "receiver": resolve_address(receiver),
                    "token": token.symbol,
                    "amount": amount,
                    "action": "transfer_in",
                    "fn_name": "Transfer",
                    "user": resolve_address(receiver),
                    "timestamp": int(tx.timeStamp)
                })

    lp_txs = pd.DataFrame(lp_txs).sort_values("timestamp")
    lp_txs["balance_change"] = lp_txs["amount"] * lp_txs["action"].map(lambda x: 1 if x in ["mint", "transfer_in"] else -1)
    lp_txs["time"] = lp_txs["timestamp"].map(unix_to_datetime)
    lp_txs = lp_txs.drop("timestamp", axis=1).set_index("time").sort_index()
    return lp_txs


def join_price(
    liquidity_txs: pd.DataFrame, 
    hourly_price: pd.Series) -> pd.DataFrame:
    """Join pric to liquidity txs"""
    if isinstance(hourly_price, pd.Series):
        hourly_price = pd.DataFrame(hourly_price)
        
    usdc_txs = liquidity_txs[liquidity_txs["token"] == Token.CUSDCLP].copy()
    weth_txs = liquidity_txs[liquidity_txs["token"] == Token.CWETHLP].copy()
    
    usdc_txs["price"] = 1.
    
    hourly_price["join_key"] = hourly_price.index.strftime("%Y%m%d%H")
    weth_txs["join_key"] = weth_txs.index.strftime("%Y%m%d%H")
    merged_weth_txs = pd.merge(
        weth_txs, 
        hourly_price, how="left", on="join_key")
    merged_weth_txs.index = weth_txs.index
    merged_weth_txs = merged_weth_txs.drop("join_key", axis=1)
    
    merged_df = pd.concat([usdc_txs, merged_weth_txs], axis=0).sort_index()
    merged_df["lp_value"] = merged_df["price"] * merged_df["amount"]
    merged_df["lp_value_change"] = merged_df["lp_value"] * merged_df["action"].map(lambda x: -1 if x in ["mint", "transfer_in"] else -1)
    
    return merged_df

In [4]:
def in_date(time: datetime, date: str) -> bool:
    min_dt = datetime.strptime(date, "%Y-%m-%d")
    max_dt = min_dt + timedelta(hours=24)
    return min_dt <= time < max_dt


def get_daily_txn(df: pd.DataFrame, chain: Chain, date: str) -> pd.DataFrame:
    """Filter transactions for each specific chain and date.
    Date should be in DD-MM-YYYY format"""
    df = df[df["chain"] == chain]
    return df[df.index.map(lambda x: in_date(x, date=date))]


def get_unique_wallets(df: pd.DataFrame) -> Dict[str, List[str]]:
    wallets = {chain: [] for chain in df["chain"].value_counts().index}

    for chain in df["chain"].value_counts().index:
        wallets[chain] = [
            _wallet for _wallet
            in df[df["chain"] == chain]["user"].value_counts().index.tolist()
            if len(_wallet) > 3]
        
    return wallets


def get_top_lp_holders_by_pool_and_chain(df: pd.DataFrame, chain: Chain, token: Token) -> pd.Series:
    is_correct_chain = df["chain"] == chain
    is_correct_token = df["token"] == token
    is_correct_fn = df["action"].isin(["mint", "burn", "transfer_in", "transfer_out"])
    df_filter = is_correct_chain & is_correct_token & is_correct_fn
    filtered_df = df[df_filter].groupby(["user"])["balance_change"].sum().sort_values(ascending=False)
    return filtered_df


def get_top_lp_holders_by_chain(
    df: pd.DataFrame, 
    chain: Chain, 
    min_usdc: float = 0.) -> pd.Series:
    is_correct_chain = df["chain"] == chain
    is_correct_fn = df["action"].isin(["mint", "burn", "transfer_in", "transfer_out"])
    df_filter = is_correct_chain & is_correct_fn
    return df[df_filter].groupby(["user"])["lp_value_change"].sum().sort_values(ascending=False)

In [53]:
def get_scores_by_chain_token(
    chain: Chain, 
    token: Token, 
    start_date: str = "2023-02-15", 
    resample_method: str = "last") -> pd.Series:
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.now()

    supported_resample = ["mean", "median", "mode", "max", "last"]
    if resample_method not in supported_resample:
        raise ValueError(f"Unknown {resample_method}, only {'|'.join(supported_resample)}")

    is_correct_chain = lp_txs["chain"] == chain
    is_correct_token = lp_txs["token"] == token
    is_correct_fn = lp_txs["action"].isin(["mint", "burn", "transfer_in", "transfer_out"])
    df_filter = is_correct_chain & is_correct_token & is_correct_fn
    filtered_txs = lp_txs[df_filter]

    wallet_stats = []
    wallets = filtered_txs["user"].unique()
    for wallet in tqdm(wallets, leave=False):
        wallet_tx = filtered_txs[
            (filtered_txs["user"] == wallet)]
        wallet_balance = wallet_tx["balance_change"].copy().cumsum()
        if ((wallet_balance.index < start_date + timedelta(days=1)) & (wallet_balance.index >= start_date)).sum() == 0:
            # no LP tx on start date
            wallet_balance.loc[start_date] = 0.
        if not max(wallet_balance.index) > datetime.strptime(end_date.strftime("%Y-%m-%d"), "%Y-%m-%d"):
            # if there's no LP provided at end_date
            wallet_balance.loc[end_date] = 0.

        if resample_method == "mean":
            wallet_balance = wallet_balance.resample("D").mean().fillna(method="ffill").cumsum()
        elif resample_method == "mode":
            wallet_balance = wallet_balance.resample("D").mode().fillna(method="ffill").cumsum()
        elif resample_method == "median":
            wallet_balance = wallet_balance.resample("D").median().fillna(method="ffill").cumsum()
        elif resample_method == "max":
            wallet_balance = wallet_balance.resample("D").max().fillna(method="ffill").cumsum()
        elif resample_method == "last":
            wallet_balance = wallet_balance.resample("D").last().fillna(method="ffill").cumsum()
        # elif resample_method == "random":
        #     pass
            # wallet_balance = wallet_balance_change.resample("D").apply(lambda x: x.sample(1)).fillna(0).cumsum()
        else:
            raise ValueError(resample_method)

        score = wallet_balance.mean()
        wallet_stats.append(score)
    wallet_scores = pd.Series(wallet_stats, index=wallets, name="LP Score").sort_values(ascending=False)
    return wallet_scores[wallet_scores > 0]


def get_wallet_stats(
    wallet: str, 
    chain: Chain, 
    token: Token, 
    start_date: str = "2023-02-15") -> pd.Series:
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.now()

    is_correct_chain = lp_txs["chain"] == chain
    is_correct_token = lp_txs["token"] == token
    is_correct_fn = lp_txs["action"].isin(["mint", "burn", "transfer_in", "transfer_out"])
    df_filter = is_correct_chain & is_correct_token & is_correct_fn
    filtered_txs = lp_txs[df_filter]

    wallet = wallet.lower().strip()
    wallet_tx = filtered_txs[(filtered_txs["user"] == wallet)]
    wallet_balance_change = wallet_tx["balance_change"].copy()
    return wallet_balance_change

In [ ]:
data_dir = "data"

api = ConnextAPI(data_dir)
transfer_api = ConnextLPTransferAPI(data_dir)
fetcher = WETHPriceFetcher(data_dir)
data = api.load_cache()
transfers = transfer_api.load_cache()

In [ ]:
from typing import Union, Optional


destination_domain = {
    6648936: Chain.ETHEREUM,
    1886350457: Chain.POLYGON,
    1869640809: Chain.OPTIMISM,
    1634886255: Chain.ARBITRUM_ONE,
    6778479: Chain.GNOSIS,
    6450786: Chain.BNB_CHAIN
}

def resolve_domain(domain_id: Union[int, str]) -> Optional[Chain]:
    if isinstance(domain_id, str):
        domain_id = int(domain_id)
    return destination_domain.get(domain_id, None)

In [7]:
watched_fn = ["xcall", "xcallIntoLocal"]

In [8]:
xcall_tx = {chain: [] for chain in api.scan_api.keys()}
for chain in xcall_tx.keys():
    for _tx in data[chain]:
        if _tx.functionName.split("(")[0] not in watched_fn:
            continue
        _tx.input["_destination"] = resolve_domain(_tx.input["_destination"])
        xcall_tx[chain].append(_tx)

In [13]:
xcall_tx[Chain.ETHEREUM][100].hash

'0x784f21ce9109cc82d7896ffe2393df37b92e5d292d11326634d2b862860d98c3'

In [14]:
xcall_tx[Chain.ETHEREUM][100].input

{'_destination': 'polygon',
 '_to': '0x1D1569E8ECb88C0cb35F8920649f1aa66173A8Bc',
 '_asset': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
 '_delegate': '0x1D1569E8ECb88C0cb35F8920649f1aa66173A8Bc',
 '_amount': 1000000000,
 '_slippage': 50,
 '_callData': ''}

In [40]:
set([tx.functionName for tx in data[Chain.OPTIMISM]])

{'',
 'addRelayer(address _relayer)',
 'addRouterLiquidityFor(uint256 _amount,address _local,address _router)',
 'addSequencer(address _sequencer)',
 'addSwapLiquidity(bytes32 key,uint256[] amounts,uint256 minToMint,uint256 deadline)',
 'approveRouter(address _router)',
 'assignRoleAdmin(address _admin)',
 'assignRoleRouterAdmin(address _router)',
 'assignRoleWatcher(address _watcher)',
 'bumpTransfer(bytes32 _transferId)',
 'disableSwap(bytes32 _key)',
 'enrollRemoteRouter(uint32 _domain,bytes32 _router)',
 'forceReceiveLocal(tuple _params)',
 'forceUpdateSlippage(tuple _params,uint256 _slippage)',
 'initializeRouter(address _owner,address _recipient)',
 'initializeSwap(bytes32 _key,address[] _pooledTokens,uint8[] decimals,string lpTokenName,string lpTokenSymbol,uint256 _a,uint256 _fee,uint256 _adminFee)',
 'proposeDiamondCut(tuple[] _diamondCut,address _init,bytes _calldata)',
 'proposeNewOwner(address newlyProposed)',
 'removeRouterLiquidity(tuple _canonical,uint256 _amount,address 

# LP Analysis 

## Loading LP Data

In [5]:
data_dir = "data"

api = ConnextAPI(data_dir)
transfer_api = ConnextLPTransferAPI(data_dir)
fetcher = WETHPriceFetcher(data_dir)
data = api.load_cache()
transfers = transfer_api.load_cache()

In [7]:
price_data = fetcher.load_cache()
price_data["date"] = price_data["unixtime"].map(unix_to_datetime)
price_data = price_data.set_index("date").sort_index()
hourly_median_price = price_data.resample("H").median()["price"]

In [8]:
blacklist_token = [
    "0x36955Fb4Ba3618d5a13701f9bb4c2d17436Ca189",  # deprecated polygon CUSDCLP
    "0x9890b51b117f765e9148A12902B0945Fa6d285E5",  # deprecated arbitrum CUSDCLP
    "0xFcc933039AC59F8F16d18B61d99D75fE60A055e3",  # deprecated BNB Chain CUSDCLP
    "0x1AF1b21323dB137603FC9eA8848053647B2C5B37",  # deprecated Gnosis CUSDCLP
    "0x0EB37a910Cb5ac05Ed85C3Be5c2Af5dAf13311B9",  # deprecated optimism CUSDCLP
]
lp_txs = get_lp_txs(data, transfers, blacklist_token=blacklist_token)
lp_txs = join_price(lp_txs, hourly_median_price)

In [9]:
lp_txs["action"].value_counts()

mint            31008
burn             6498
transfer_in      1131
transfer_out     1131
Name: action, dtype: int64

In [10]:
dates = list({_item.strftime("%Y-%m-%d") for _item in lp_txs.index})
wallets = get_unique_wallets(lp_txs)

In [11]:
{_chain: len(addrs) for _chain, addrs in wallets.items()}

{'arbitrum_one': 11565,
 'polygon': 4858,
 'optimism': 2368,
 'bnb_chain': 1370,
 'gnosis': 1036}

Number of unique addresses for each chain

In [12]:
threshold = 0.3

### Ranking with averages (using TWAP scores)

In [28]:
query = "0xCEdf866187680e70296a8b9EEce22A09b4F05798".lower()

In [54]:
for chain in wallets.keys():
    print(f"Chain: {chain}")
    for token in [Token.CWETHLP, Token.CUSDCLP]:
        scores = get_scores_by_chain_token(
            chain=chain, token=token, resample_method="last")
        n_qualified = round(threshold * len(tmp))
        qualified = scores.iloc[:n_qualified]
        min_score = qualified.values[-1]
        
        print(f"\t{token} ({n_qualified}/{len(scores)}) :: min_score {min_score}")

Chain: arbitrum_one


  0%|          | 0/6314 [00:00<?, ?it/s]

	CWETHLP (1391/6306) :: min_score 1.23992614435093


  0%|          | 0/5845 [00:00<?, ?it/s]

	CUSDCLP (1391/5724) :: min_score 8948.178203363865
Chain: polygon


...

	CUSDCLP (1391/1233) :: min_score 0.00035000000000000016
Chain: gnosis


  0%|          | 0/65 [00:00<?, ?it/s]

	CWETHLP (1391/64) :: min_score 4.588059701492538e-05


  0%|          | 0/994 [00:00<?, ?it/s]

	CUSDCLP (1391/985) :: min_score 1.2210526315789477e-05


### Ranking without averages

In [195]:
# min_value_usdc = 0.
# min_value_weth = 0.

min_value_usdc = 10.
min_value_weth = 0.001

epsilon = 1e-10  # floating point error

print(f"Applying filter with minimum USDC of {min_value_usdc}")
print(f"Applying filter with minimum WETH of {min_value_weth}\n")

for chain in wallets.keys():
    print(f"Chain: {chain}")
    for token in [Token.CWETHLP, Token.CUSDCLP]:
        df = get_top_lp_holders_by_pool_and_chain(lp_txs, chain, token)
        df = df[df > epsilon]
        n_ori_lp = len(df)
        if token == Token.CUSDCLP:
            df = df[df >= min_value_usdc]
        elif token == Token.CWETHLP:
            df = df[df >= min_value_weth]
        n_filtered_lp = len(df)
        candidates = df.iloc[:round(threshold * len(df))]
        min_value = candidates.values[-1]
        print(f"Lowest {token} amount to reach top 30% [{len(candidates)}/{n_ori_lp}] : {min_value}")
        
        n_removed = n_ori_lp - n_filtered_lp
        if n_removed > 0:
            print(f"  {n_removed} ({n_removed*100/n_ori_lp:.2f}%) holders are removed for providing less than minimum amount")
    print()

Applying filter with minimum USDC of 10.0
Applying filter with minimum WETH of 0.001

Chain: arbitrum_one
Lowest CWETHLP amount to reach top 30% [832/5907] : 0.3205321513419961
  3132 (53.02%) holders are removed for providing less than minimum amount
Lowest CUSDCLP amount to reach top 30% [859/4341] : 1066.106072987634
  1479 (34.07%) holders are removed for providing less than minimum amount

Chain: polygon
Lowest CWETHLP amount to reach top 30% [46/188] : 1.0251056540850008
  33 (17.55%) holders are removed for providing less than minimum amount
Lowest CUSDCLP amount to reach top 30% [584/3600] : 1032.8729423062655
  1655 (45.97%) holders are removed for providing less than minimum amount

Chain: optimism
Lowest CWETHLP amount to reach top 30% [80/322] : 1.026323028733804
  55 (17.08%) holders are removed for providing less than minimum amount
Lowest CUSDCLP amount to reach top 30% [328/1572] : 1551.215864054076
  477 (30.34%) holders are removed for providing less than minimum amou

### Calculating Scores

In [298]:
wallet = "0x26daee818a1e38a46825fafb8a729a1c021cc726".lower()
# wallet = "0x0a9a04f289e1237c978d9ed375eeebda73ee97b1"
# wallet = "0x13daf15e66e6214a1645413f3d858d3947f5900a".lower()
chain = Chain.POLYGON
token = Token.CUSDCLP

In [299]:
min_filter = 0.001 if token == Token.CWETHLP else 10.
# min_filter = 0.

df = get_top_lp_holders_by_pool_and_chain(lp_txs, chain, token)
df = df[df > min_filter]
df = df[:round(len(df) * threshold)]

for i, (_addr, _amt) in enumerate(df.items()):
    if _addr == wallet.lower():
        print(f"Wallet {wallet}:")
        print(f"Total of {len(df)} wallets, you are at rank {i+1} (Top {((i+1)*100/len(df)):.2f}%)")
        print(f"Liquidity provided amount: {_amt} {token} (minimum at {df.values[-1]} {token})")
        if min_filter > 0:
            print(f"\n>>NOTE: Applying filter at {min_filter} {token}")

Wallet 0x26daee818a1e38a46825fafb8a729a1c021cc726:
Total of 584 wallets, you are at rank 9 (Top 1.54%)
Liquidity provided amount: 11702.100333601102 CUSDCLP (minimum at 1032.8729423062655 CUSDCLP)

>>NOTE: Applying filter at 10.0 CUSDCLP


# `xcall` Analysis